In [1]:
import tensorflow as tf
import numpy as np
import pickle
max_batch = 200
max_step = 40
feature = 200

def padding(filename):
    train_data = pickle.load(open(filename, 'rb'))
    train_inputs = train_data['inputs']
    train_targets = train_data['targets']
    print(train_inputs['13_0'].shape)
    print(train_targets['13_0'].shape)
    padded_inputs = dict()
    padded_targets = dict()
    for key in train_inputs.keys():
        temp = train_inputs[key]
        temp2 = train_targets[key]
        if temp.shape[0] != max_batch and temp.shape[0] != 0:
    #         print(temp.shape)
    #         print(50-len(temp))
    #         print(np.zeros((50 - temp.shape[0], temp.shape[1], 200)).shape)
            temp = np.concatenate((temp, np.zeros((max_batch - temp.shape[0], temp.shape[1], feature))), 0)
            temp2 = np.concatenate((temp2, np.zeros((max_batch - temp2.shape[0], 134))), 0)
    #         print(temp.shape)
            temp = np.concatenate((temp, np.zeros((max_batch, max_step - temp.shape[1], feature))), 1)
            padded_inputs[key] = temp
            padded_targets[key] = temp2
        elif temp.shape[0] == max_batch and temp.shape[0] != 0:
            temp = np.concatenate((temp, np.zeros((max_batch, max_step - temp.shape[1], feature))), 1)
            padded_inputs[key] = temp
            padded_targets[key] = train_targets[key]
    print(padded_inputs['13_1'].shape)
    print(padded_targets['13_1'].shape)
    return padded_inputs, padded_targets
    
padded_inputs, padded_targets = padding('../word2vec/anouymous_train_data_200.npz')
padded_test_inputs, padded_test_targets  = padding('../word2vec/anouymous_valid_data_200.npz')

def init(shape):
    return tf.Variable(tf.truncated_normal(shape), tf.float32)

def lstm_layer(inputs, batch_size, step_size):
    shape = inputs.shape.as_list()
    batch_size = shape[0]
    step_size = shape[1]
    n_units = shape[2]
#     print(batch_size)
#     print(step_size)

    wf = init([n_units, n_units])
    bf = init([n_units])

    wi = init([n_units, n_units])
    bi = init([n_units])

    wc = init([n_units, n_units])
    bc = init([n_units])

    wo = init([n_units, n_units])
    bo = init([n_units])

    ht = init([batch_size, n_units])
    ct = init([1, n_units])

    ct_tiled = tf.tile(ct, [batch_size, 1])
    
    W = tf.transpose(tf.tile(tf.concat([wf, wi, wc, wo], 0), [1, 2]), [1, 0])
    B = tf.concat([bf, bi, bc, bo], 0)
    
        
    for i in range(step_size):
        xt = inputs[:, i, :]
#         ft = tf.sigmoid(tf.matmul(ht, wf) + tf.matmul(xt, wf) + bf)
#         it = tf.sigmoid(tf.matmul(ht, wi) + tf.matmul(xt, wi) + bi)
#         c = tf.tanh(tf.matmul(ht, wc) + tf.matmul(xt, wc) + bc)
#         ot = tf.sigmoid(tf.matmul(ht, wo) + tf.matmul(xt, wo) + bo)
        X = tf.concat([ht, xt], 1)
#         print(ht.shape)
#         print(xt.shape)
#         print(X.shape)
        V = tf.sigmoid(tf.matmul(X, W) + B)
#         print(W.shape)
#         print(B.shape)
        ft, it, c, ot = tf.split(V, num_or_size_splits=4, axis=1)
#         print(ft.shape)
#         print(it.shape)
#         print(c.shape)
#         print(ot.shape)
        
        ct = tf.reshape(tf.reduce_mean(ft * ct_tiled + it * c, 0), [1, n_units])
        ht = ot * tf.tile(tf.tanh(ct), [batch_size, 1])
        
    return ht, ct

In [69]:
graph = tf.Graph()
with graph.as_default():
    
    batch_size = max_batch
    step_size = max_step
    
    inputs_placeholder = tf.placeholder(tf.float32, [batch_size, step_size, feature])
    targets_placeholder = tf.placeholder(tf.float32, [batch_size, 134])
#     batch_size = tf.placeholder(tf.int32)
#     step_size = tf.placeholder(tf.int32)
    
#     outputs, states = tf.nn.dynamic_rnn(
#             cell = tf.contrib.rnn.BasicLSTMCell(num_units=200),
#             inputs = inputs_placeholder,
#             dtype = tf.float32)
        
#     print(outputs)
#     print(cells)
    outputs, states = lstm_layer(inputs_placeholder, batch_size, step_size)
    
    w1 = init([200, 300])
    b1 = init([300])
    
    w2 = init([300, 300])
    b2 = init([300])
    
    w3 = init([300, 134])
    b3 = init([134])
    
    o1 = tf.nn.relu(tf.add(tf.matmul(outputs, w1), b1))
    o2 = tf.nn.relu(tf.add(tf.matmul(o1, w2), b2))
    o3 = tf.nn.relu(tf.add(tf.matmul(o2, w3), b3))
    
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=o3, labels=targets_placeholder))
    sum_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=o3, labels=targets_placeholder))
    optimizer = tf.train.AdamOptimizer().minimize(loss)
    accuracy = tf.reduce_sum(tf.cast(tf.equal(tf.argmax(o3, 1), tf.argmax(targets_placeholder, 1)), tf.float32))
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    keys = padded_inputs.keys()
    test_keys = padded_test_inputs.keys()
    
    for i in range(100):
        accs = 0.0
        errs = 0.0
        n = 0
        for key in keys:
            batch_inputs = padded_inputs[key]
            batch_targets = padded_targets[key]
            batch_size = len(train_inputs)
#             step_size = key.split('_')[0]
#             print(batch_inputs.shape)
#             print(batch_targets.shape)
            n += batch_size
            feed_dict = {inputs_placeholder:batch_inputs, targets_placeholder:batch_targets}#, batch_size:batch_size, step_size:step_size}
            _, acc, err = sess.run([optimizer, accuracy, sum_loss], feed_dict=feed_dict)
            accs += acc
            errs += err
        print('#Trainig Epoch: %d, ACC:%f ERR:%f'%(i, accs/n, errs/n))
        if i % 1 == 0:
            accs = 0.0
            errs = 0.0
            n = 0
            for key in keys:
                batch_inputs = padded_test_inputs[key]
                batch_targets = padded_test_targets[key]
                batch_size = len(train_inputs)
    #             step_size = key.split('_')[0]
    #             print(batch_inputs.shape)
    #             print(batch_targets.shape)
                n += batch_size
                feed_dict = {inputs_placeholder:batch_inputs, targets_placeholder:batch_targets}#, batch_size:batch_size, step_size:step_size}
                _, acc, err = sess.run([optimizer, accuracy, sum_loss], feed_dict=feed_dict)
                accs += acc
                errs += err
            print('#Trainig Epoch: %d, ACC:%f ERR:%f'%(i, accs/n, errs/n))
            

#Trainig Epoch: 0, ACC:0.000000 ERR:7521.940202
#Trainig Epoch: 0, ACC:0.000000 ERR:7445.793588
#Trainig Epoch: 0, ACC:0.000000 ERR:6970.277378
#Trainig Epoch: 0, ACC:0.000000 ERR:6611.534132
#Trainig Epoch: 0, ACC:0.000000 ERR:6587.760447
#Trainig Epoch: 0, ACC:0.005764 ERR:6450.528458
#Trainig Epoch: 0, ACC:0.004940 ERR:6358.735900
#Trainig Epoch: 0, ACC:0.004323 ERR:6327.801513
#Trainig Epoch: 0, ACC:0.003842 ERR:6199.584774
#Trainig Epoch: 0, ACC:0.003458 ERR:6118.193624
#Trainig Epoch: 0, ACC:0.003144 ERR:5948.444754
#Trainig Epoch: 0, ACC:0.002882 ERR:5872.259396
#Trainig Epoch: 0, ACC:0.002660 ERR:5799.019258
#Trainig Epoch: 0, ACC:0.002470 ERR:5723.075983
#Trainig Epoch: 0, ACC:0.002305 ERR:5655.760159
#Trainig Epoch: 0, ACC:0.005403 ERR:5559.403976
#Trainig Epoch: 0, ACC:0.005086 ERR:5462.422084
#Trainig Epoch: 0, ACC:0.004803 ERR:5348.976885
#Trainig Epoch: 0, ACC:0.004550 ERR:5272.461152
#Trainig Epoch: 0, ACC:0.004323 ERR:5186.809456
#Trainig Epoch: 0, ACC:0.004117 ERR:5125

#Trainig Epoch: 0, ACC:0.001977 ERR:1417.053749
#Trainig Epoch: 0, ACC:0.001966 ERR:1409.233975
#Trainig Epoch: 0, ACC:0.001954 ERR:1401.386883
#Trainig Epoch: 0, ACC:0.001943 ERR:1393.750079
#Trainig Epoch: 0, ACC:0.001932 ERR:1386.685218
#Trainig Epoch: 0, ACC:0.001921 ERR:1379.019466
#Trainig Epoch: 0, ACC:0.001910 ERR:1372.054265
#Trainig Epoch: 0, ACC:0.001900 ERR:1365.245520
#Trainig Epoch: 0, ACC:0.001889 ERR:1358.127199
#Trainig Epoch: 0, ACC:0.001879 ERR:1351.131801
#Trainig Epoch: 0, ACC:0.001868 ERR:1343.946884
#Trainig Epoch: 0, ACC:0.001858 ERR:1337.213872
#Trainig Epoch: 0, ACC:0.001848 ERR:1330.231272
#Trainig Epoch: 0, ACC:0.001838 ERR:1323.272325
#Trainig Epoch: 0, ACC:0.001828 ERR:1316.425692
#Trainig Epoch: 0, ACC:0.001818 ERR:1309.848967
#Trainig Epoch: 0, ACC:0.001809 ERR:1303.528311
#Trainig Epoch: 0, ACC:0.001799 ERR:1297.181567
#Trainig Epoch: 0, ACC:0.001790 ERR:1290.551934
#Trainig Epoch: 0, ACC:0.001780 ERR:1283.935261
#Trainig Epoch: 0, ACC:0.001771 ERR:1277

#Trainig Epoch: 0, ACC:0.003517 ERR:733.592014
#Trainig Epoch: 0, ACC:0.003507 ERR:731.549168
#Trainig Epoch: 0, ACC:0.004045 ERR:729.518096
#Trainig Epoch: 0, ACC:0.004033 ERR:727.498697
#Trainig Epoch: 0, ACC:0.004021 ERR:725.490871
#Trainig Epoch: 0, ACC:0.004055 ERR:723.494518
#Trainig Epoch: 0, ACC:0.004044 ERR:721.509540
#Trainig Epoch: 0, ACC:0.004032 ERR:719.569883
#Trainig Epoch: 0, ACC:0.004021 ERR:717.607269
#Trainig Epoch: 0, ACC:0.004009 ERR:715.655744
#Trainig Epoch: 0, ACC:0.003998 ERR:713.715213
#Trainig Epoch: 0, ACC:0.003987 ERR:711.788062
#Trainig Epoch: 0, ACC:0.003976 ERR:709.869237
#Trainig Epoch: 0, ACC:0.003965 ERR:707.961131
#Trainig Epoch: 0, ACC:0.003954 ERR:706.109876
#Trainig Epoch: 0, ACC:0.003943 ERR:704.248518
#Trainig Epoch: 0, ACC:0.003932 ERR:702.371838
#Trainig Epoch: 0, ACC:0.003921 ERR:700.526168
#Trainig Epoch: 0, ACC:0.003910 ERR:698.734004
#Trainig Epoch: 0, ACC:0.003899 ERR:696.887942
#Trainig Epoch: 0, ACC:0.003889 ERR:695.051995
#Trainig Epoc

#Trainig Epoch: 0, ACC:0.004447 ERR:497.947143
#Trainig Epoch: 0, ACC:0.004439 ERR:497.043877
#Trainig Epoch: 0, ACC:0.004430 ERR:496.207282
#Trainig Epoch: 0, ACC:0.004422 ERR:495.309692
#Trainig Epoch: 0, ACC:0.004414 ERR:494.415527
#Trainig Epoch: 0, ACC:0.004405 ERR:493.524768
#Trainig Epoch: 0, ACC:0.004397 ERR:492.637397
#Trainig Epoch: 0, ACC:0.004388 ERR:491.753393
#Trainig Epoch: 0, ACC:0.004380 ERR:490.872738
#Trainig Epoch: 0, ACC:0.004372 ERR:489.995412
#Trainig Epoch: 0, ACC:0.004364 ERR:489.121396
#Trainig Epoch: 0, ACC:0.004355 ERR:488.250673
#Trainig Epoch: 0, ACC:0.004347 ERR:487.383223
#Trainig Epoch: 0, ACC:0.004339 ERR:486.519028
#Trainig Epoch: 0, ACC:0.004331 ERR:485.658070
#Trainig Epoch: 0, ACC:0.004323 ERR:484.801723
#Trainig Epoch: 0, ACC:0.004315 ERR:483.947181
#Trainig Epoch: 0, ACC:0.004307 ERR:483.095822
#Trainig Epoch: 0, ACC:0.004299 ERR:482.247628
#Trainig Epoch: 0, ACC:0.004291 ERR:481.402581
#Trainig Epoch: 0, ACC:0.004283 ERR:480.560664
#Trainig Epoc

#Trainig Epoch: 1, ACC:0.000000 ERR:30.137814
#Trainig Epoch: 1, ACC:0.000000 ERR:28.452575
#Trainig Epoch: 1, ACC:0.000000 ERR:27.890829
#Trainig Epoch: 1, ACC:0.000000 ERR:27.609955
#Trainig Epoch: 1, ACC:0.000000 ERR:27.441432
#Trainig Epoch: 1, ACC:0.000000 ERR:27.329082
#Trainig Epoch: 1, ACC:0.000000 ERR:27.248833
#Trainig Epoch: 1, ACC:0.000000 ERR:27.188646
#Trainig Epoch: 1, ACC:0.000000 ERR:27.141834
#Trainig Epoch: 1, ACC:0.000000 ERR:27.104384
#Trainig Epoch: 1, ACC:0.000000 ERR:27.073743
#Trainig Epoch: 1, ACC:0.000000 ERR:27.280986
#Trainig Epoch: 1, ACC:0.000000 ERR:27.774529
#Trainig Epoch: 1, ACC:0.000000 ERR:27.702587
#Trainig Epoch: 1, ACC:0.000000 ERR:27.640237
#Trainig Epoch: 1, ACC:0.000000 ERR:27.585681
#Trainig Epoch: 1, ACC:0.000000 ERR:27.537543
#Trainig Epoch: 1, ACC:0.000000 ERR:27.494753
#Trainig Epoch: 1, ACC:0.000000 ERR:27.456468
#Trainig Epoch: 1, ACC:0.000000 ERR:27.422012
#Trainig Epoch: 1, ACC:0.000000 ERR:27.390837
#Trainig Epoch: 1, ACC:0.000000 ER

...

#Trainig Epoch: 1, ACC:0.006819 ERR:27.895253
#Trainig Epoch: 1, ACC:0.006806 ERR:27.893160
#Trainig Epoch: 1, ACC:0.006794 ERR:27.891075
#Trainig Epoch: 1, ACC:0.006781 ERR:27.888998
#Trainig Epoch: 1, ACC:0.006769 ERR:27.886929
#Trainig Epoch: 1, ACC:0.006756 ERR:27.884867
#Trainig Epoch: 1, ACC:0.006744 ERR:27.882812
#Trainig Epoch: 1, ACC:0.006731 ERR:27.880766
#Trainig Epoch: 1, ACC:0.006719 ERR:27.878726
#Trainig Epoch: 1, ACC:0.006707 ERR:27.876695
#Trainig Epoch: 1, ACC:0.006695 ERR:27.874670
#Trainig Epoch: 1, ACC:0.006682 ERR:27.872653
#Trainig Epoch: 1, ACC:0.006670 ERR:27.870644
#Trainig Epoch: 1, ACC:0.006658 ERR:27.868641
#Trainig Epoch: 1, ACC:0.006646 ERR:27.866646
#Trainig Epoch: 1, ACC:0.006634 ERR:27.864658
#Trainig Epoch: 1, ACC:0.006622 ERR:27.862677
#Trainig Epoch: 1, ACC:0.006610 ERR:27.860704
#Trainig Epoch: 1, ACC:0.006598 ERR:27.858737
#Trainig Epoch: 1, ACC:0.006586 ERR:27.856778
#Trainig Epoch: 1, ACC:0.006575 ERR:27.854826
#Trainig Epoch: 1, ACC:0.006563 ER

#Trainig Epoch: 2, ACC:0.000000 ERR:26.767336
#Trainig Epoch: 2, ACC:0.000000 ERR:26.767336
#Trainig Epoch: 2, ACC:0.000000 ERR:26.767336
#Trainig Epoch: 2, ACC:0.000000 ERR:26.767336
#Trainig Epoch: 2, ACC:0.000000 ERR:26.767336
#Trainig Epoch: 2, ACC:0.000000 ERR:26.767336
#Trainig Epoch: 2, ACC:0.000000 ERR:26.767336
#Trainig Epoch: 2, ACC:0.007157 ERR:26.767336
#Trainig Epoch: 2, ACC:0.006926 ERR:26.767336
#Trainig Epoch: 2, ACC:0.006709 ERR:26.767336
#Trainig Epoch: 2, ACC:0.006506 ERR:26.767336
#Trainig Epoch: 2, ACC:0.006315 ERR:26.767336
#Trainig Epoch: 2, ACC:0.006134 ERR:26.767336
#Trainig Epoch: 2, ACC:0.005964 ERR:26.767336
#Trainig Epoch: 2, ACC:0.005803 ERR:26.767336
#Trainig Epoch: 2, ACC:0.005650 ERR:26.767336
#Trainig Epoch: 2, ACC:0.005505 ERR:26.767336
#Trainig Epoch: 2, ACC:0.005367 ERR:26.767336
#Trainig Epoch: 2, ACC:0.005237 ERR:26.767336
#Trainig Epoch: 2, ACC:0.005112 ERR:26.767336
#Trainig Epoch: 2, ACC:0.008277 ERR:26.767336
#Trainig Epoch: 2, ACC:0.008089 ER

#Trainig Epoch: 2, ACC:0.006484 ERR:28.192256
#Trainig Epoch: 2, ACC:0.006452 ERR:28.185236
#Trainig Epoch: 2, ACC:0.006421 ERR:28.178286
#Trainig Epoch: 2, ACC:0.006389 ERR:28.171403
#Trainig Epoch: 2, ACC:0.006358 ERR:28.164587
#Trainig Epoch: 2, ACC:0.006328 ERR:28.157837
#Trainig Epoch: 2, ACC:0.006297 ERR:28.151152
#Trainig Epoch: 2, ACC:0.006267 ERR:28.144531
#Trainig Epoch: 2, ACC:0.006237 ERR:28.137973
#Trainig Epoch: 2, ACC:0.006208 ERR:28.131477
#Trainig Epoch: 2, ACC:0.006178 ERR:28.125043
#Trainig Epoch: 2, ACC:0.006149 ERR:28.118668
#Trainig Epoch: 2, ACC:0.006121 ERR:28.112354
#Trainig Epoch: 2, ACC:0.006092 ERR:28.106098
#Trainig Epoch: 2, ACC:0.006064 ERR:28.121988
#Trainig Epoch: 2, ACC:0.006036 ERR:28.115746
#Trainig Epoch: 2, ACC:0.006008 ERR:28.109560
#Trainig Epoch: 2, ACC:0.005981 ERR:28.103431
#Trainig Epoch: 2, ACC:0.005954 ERR:28.097358
#Trainig Epoch: 2, ACC:0.007048 ERR:28.091340
#Trainig Epoch: 2, ACC:0.007016 ERR:28.085376
#Trainig Epoch: 2, ACC:0.006985 ER

#Trainig Epoch: 2, ACC:0.007636 ERR:27.552410
#Trainig Epoch: 2, ACC:0.007616 ERR:27.550355
#Trainig Epoch: 2, ACC:0.007596 ERR:27.548310
#Trainig Epoch: 2, ACC:0.007576 ERR:27.546276
#Trainig Epoch: 2, ACC:0.007556 ERR:27.544253
#Trainig Epoch: 2, ACC:0.007537 ERR:27.542240
#Trainig Epoch: 2, ACC:0.007517 ERR:27.540238
#Trainig Epoch: 2, ACC:0.007498 ERR:27.538246
#Trainig Epoch: 2, ACC:0.007479 ERR:27.536264
#Trainig Epoch: 2, ACC:0.007460 ERR:27.534293
#Trainig Epoch: 2, ACC:0.007440 ERR:27.532331
#Trainig Epoch: 2, ACC:0.007421 ERR:27.530380
#Trainig Epoch: 2, ACC:0.007403 ERR:27.528438
#Trainig Epoch: 2, ACC:0.007384 ERR:27.526506
#Trainig Epoch: 2, ACC:0.007365 ERR:27.524584
#Trainig Epoch: 2, ACC:0.007347 ERR:27.522672
#Trainig Epoch: 2, ACC:0.007328 ERR:27.520769
#Trainig Epoch: 2, ACC:0.007310 ERR:27.518876
#Trainig Epoch: 2, ACC:0.007291 ERR:27.516993
#Trainig Epoch: 2, ACC:0.007273 ERR:27.515119
#Trainig Epoch: 2, ACC:0.007255 ERR:27.513254
#Trainig Epoch: 2, ACC:0.007237 ER

KeyboardInterrupt: 